# 0. Carregando os dados e bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from matplotlib import gridspec #Para fazer os dashs
#Foram adicionados por conta de erro de renderização no mapa (ex1.10), utilizando o VS Code. Caso sua IDE não tenha o erro, não é necessário
#Erro: No renderer could be found for mimetype "application/vnd.plotly.v1+json"
import plotly.io as pio
pio.renderers.default = "notebook_connected"
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from ipywidgets import fixed
import time
from multiprocessing import Pool

In [ ]:
data = pd.read_csv('datasets/kc_house_data.csv')

# 1. Exercícios

## 1.1 Organize o código do módulo 4 em funções

## 1.2 Refaça o gráfico e o dash com o Streamlit